### 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
### 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

目標網址：https://www.ettoday.net/

觀察新聞類別和網址之間的關係：<br>
社會新聞，網址為：https://www.ettoday.net/news/focus/社會/ (https://www.ettoday.net/news/focus/%E7%A4%BE%E6%9C%83/)<br>
政治新聞，網址為：https://www.ettoday.net/news/focus/政治/ (https://www.ettoday.net/news/focus/%E6%94%BF%E6%B2%BB/)<br>

In [123]:
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import requests

In [124]:
keyword = '政治'
utf8_url = repr(keyword.encode('utf-8')).upper()
utf8_url = utf8_url.replace('\\X','%')
url = 'https://www.ettoday.net/news/focus/'+utf8_url[2:-1]+'/'
url

'https://www.ettoday.net/news/focus/%E6%94%BF%E6%B2%BB/'

In [125]:
options = Options()
options.add_argument('--disable-notifications')

browser = webdriver.Chrome(executable_path='./chromedriver',options = options)
browser.get(url)

In [126]:
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [127]:
html = browser.page_source
soup = BeautifulSoup(html,'lxml')
browser.close()

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [128]:
#定義一個可以印出外部連結內文的函數
def extra_link_concent(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'lxml')
    content = soup.find('div',class_="story",itemprop="articleBody").find_all('p')
    for c in content:
        if c.string is not None:
            print(c.string)
    

In [130]:
all_news = soup.find('div',class_="part_pictxt_3 lazyload").find_all('div',class_="piece clearfix")
for i in all_news:
    title = i.find('a').get('title')
    summary = i.find(class_='summary').text
    news_url = 'https://www.ettoday.net'+i.find('a').get('href')
    print('標題：',title)
    print('摘要：',summary)
    print('內文')
    extra_link_concent(news_url)
    print('----------------')

標題： 陳時中曾讚「真指揮官」！學者齊聲挺陳其邁防疫貢獻
摘要： 國民黨高雄市長候選人李眉蓁碩士論文遭爆高達96％抄襲，引發政壇論文大戰。國民黨方面對李眉蓁稱要「放棄」碩士學位讚賞勇於認錯，同時又點名包含總統蔡英文、曾任行政院副院長的陳其邁論文也有問題。對此，台大公..
內文
▲陳其邁  。（圖／記者李毓康攝）
記者蔣婕妤／台北報導
國民黨高雄市長候選人李眉蓁碩士論文遭爆高達96％抄襲，引發政壇論文大戰。國民黨方面對李眉蓁稱要「放棄」碩士學位讚賞勇於認錯，同時又點名包含總統蔡英文、曾任行政院副院長的陳其邁論文也有問題。對此，台大公共衛生學院院長詹長權及健康保險署長李伯璋力挺陳其邁是「主要貢獻者」，臉書粉專高雄好過日酸抹黑者，「抹黑陳其邁，其實凸顯其邁真本事。」
李眉蓁被踢爆碩士論文幾乎「整本抄襲」，第一時間召開記者會回擊的她，事隔一日出面爆哭稱要放棄碩士學位。不少藍營人士竟讚賞李眉蓁，反而質疑陳其邁在擔任行政院副院長時，與12名專家學者共同發表的防疫論文為何可以列為「第一作者」。
藍營質疑的這本論文，正是今年5月、各國疫情正險峻時，由陳其邁主導與疾管署、資安處、健保署、台大公共衛生學院等跨部門共同合作，共計12位國內知名學者共同撰寫發表，並以鑽石公主號防疫過程為主題的論文，投稿國際知名的「醫學網路研究期刊」（Journal of Medical Internet Research）也獲刊登。
國立台灣大學公共衛生學院院長詹長權也表示力挺，他回憶陳其邁從二月初投入防疫工作以來，常常跟自己討論面對疫情如何運用科技解決問題，並逐步提出不同想法，「他整合政府跨部會、跨部門，因為這牽涉到交通部門、內政部門、衛福部門等，這會幫台灣建構一個非常好的數位防疫的系統，這整個東西都是他去完成的。」
詹長權回憶，「這個過程，他（陳其邁）想說這麼好的防疫措施，應該好好留下來，可以給世界各國的人用，他想說寫成論文，我也覺得不錯，『他就是最主要的貢獻者』。」
中央健康保險署長李伯璋也指出，防疫論文從計畫及研究的架構、設計、執行、起草，都是陳其邁負責，強調陳其邁是主要貢獻者，掛名論文第一位是「名符其實」。
陳其邁在防疫期間複雜跨部門協調，衛福部長陳時中7月初曾表示，自己是防疫前線的指揮官，不過許多防疫的事情要呈報行政院執行，而這些是陳其邁擔任行政院副院長時負責的，陳其邁在幕後默默做，「他才是真正

### .text和.string的不同：
* .text<br>
不論tag有幾個，會把所有的內文取出。<br>
* .string<br>
如果tag只有一個，會把內文取出。<br>
若是tag超過一個，會不知道取哪一個tag的內文，所以返回None

### document.body.scrollHeight <br>
指的是DOM裡body的視窗文件長度(不論滾論) <br>
如下面所見：<br>
一開始開網頁可見的內容高度為6464 <br>
再往下滑一次滾輪到底，內容高度為 8872 <br>
將新高度取代原來高度，以此類推，只要滑到文章末，新高度 = 原來高度，代表已經滑到最底了。

In [50]:
last_height = browser.execute_script("return document.body.scrollHeight")
print(last_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
new_height = browser.execute_script("return document.body.scrollHeight")
print(new_height)

6464
8872
11410
13915
16284
18745
21151
23612
25100
25150


在跨瀏覽器下，若要取得HTML文件寬高，<br>
可以在document.documentElement上以scrollWidth、scrollHeight取得。<br>
<br>
若要取得body寬高，<br>
則可以使用document.body的scrollWidth、scrollHeight取得。<br>
<br>
若要取得螢幕的寬高，<br>
可以使用screen的width、height取得。<br>
<br>
若要取得螢幕可用區域的寬高，不含工具列的範圍，<br>
可在screen的availWidth、availHeight取得。<br>
<br>
在遵守標準的瀏覽器上，以下相關的資訊都可以在window物件上取得：<br>
* screenX、screenY：瀏覽器視窗在螢幕中的位置
* outerWidth、outerHeight：瀏覽器視窗寬、高
* innerWidth、innerHeight：顯示視埠區域寬高（不包括選單、工具列、捲軸）
* pageXOffset、pageYOffset：水平、垂直捲軸位置